# Six Hats Solver

The Six Hats Solver uses intelligent autonomous agents inspired by Edward de Bono’s Six Thinking Hats framework. While this project draws conceptual inspiration from de Bono’s work, it is an independent software implementation and is not an official, endorsed, or fully faithful reproduction of the original methodology. Each “hat” functions as a specialized cognitive lens, enabling richer, more diverse, and more actionable insights than a single-model response. This system enhances decision‑making by integrating critical reasoning, creativity, emotional awareness, and structured synthesis.

![Six Hats Solver Banner](../images/SixHatsSolver.png) 

| Hat        | Role                      |
|-----------|---------------------------|
| ⬜ White   | Facts & Information       |
| 🟥 Red     | Feelings & Intuition      |
| ⬛ Black   | Caution & Critical        |
| 🟨 Yellow | Benefits & Optimism       |
| 🟩 Green  | Creativity & New Ideas    |
| 🟦 Blue   | Process & Control         |


## 1. Environment Setup & Imports ⚙️

In this section we load all required Python packages, agent framework components, and custom hat factories needed to run the Six Hats Solver.


In [1]:
import os
import warnings

from dotenv import load_dotenv
from google.adk.agents import ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.adk.runners import InMemoryRunner
from google.genai import types
from IPython.display import Markdown, display

from agents_intensive_capstone.agents import (
    black_hat_factory,
    blue_hat_factory,
    green_hat_factory,
    red_hat_factory,
    white_hat_factory,
    yellow_hat_factory,
)

warnings.filterwarnings("ignore")
print("✅ Environment Setup & Imports complete.")

c:\Praveen\Personal\git\agents-intensive-capstone-2025\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ Environment Setup & Imports complete.


## 2. API Credentials & Retry Strategy 🔐

Here we load the Gemini API key from the environment and define robust HTTP retry options to make the solver resilient to transient API failures.

> 🔁 **Resilience Note:** We retry on 429, 500, 503, and 504 to handle rate limits and transient server errors.



In [2]:
load_dotenv()

# 2. Access the variable
api_key = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key setup complete.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.


## 3. LLM Model Initialization 🤖

We instantiate the Gemini model (`gemini-2.5-flash-lite`) with the configured retry options. This model powers all Six Hats agents.


In [3]:
gemini_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

## 4. Six Hats Agent Configuration 🎩

Each hat is implemented as a specialized agent sharing the same underlying Gemini model but optimized for a different thinking style.


### **4.1 White Hat -  Facts & Information** ⬜

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#f0f0f0" />
  <rect x="30" y="60" width="40" height="15" fill="#000000" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#f0f0f0" />
</svg>

The White Hat represents a neutral, information‑driven perspective. It focuses entirely on factual material—what is known, what is verified, and what data is missing. The purpose of this hat is to establish a grounded baseline so that all further reasoning begins from shared, objective reality.

Key aspects include:

* Identifying available data and evidence
* Highlighting gaps in knowledge or missing information
* Clarifying assumptions that need validation
* Distinguishing between fact, belief, and interpretation

This hat ensures the discussion remains anchored in reliable information before diving into opinions or creative thinking.



In [4]:
white_hat = white_hat_factory.WhiteHatFactory.create(
    model=gemini_model
)

### **4.2 Red Hat — Emotions & Instincts**   🟥
<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#e74c3c" />
  <rect x="30" y="60" width="40" height="15" fill="#000000" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#e74c3c" />
</svg>

 The Red Hat provides space for emotional responses, gut instincts, and intuitive impressions. These reactions are valid and informative, even when they cannot be logically explained.

Typical contributions include:

* Immediate emotional reactions to an idea
* Feelings of excitement, hesitation, or concern
* Intuitive insights that may hint at deeper issues
* Emotional patterns that might influence acceptance or resistance

This hat prevents emotional undercurrents from remaining unspoken and ensures they are considered as part of the decision‑making process.

In [5]:
red_hat = red_hat_factory.RedHatFactory.create(model=gemini_model)
print("✅ Red Hat configured")

✅ Red Hat configured


### **4.3 Black Hat — Risks & Critical Thinking**  ⬛

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#000000" />
  <rect x="30" y="60" width="40" height="15" fill="#f0f0f0" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#000000" />
</svg>

The Black Hat embodies caution, skepticism, and critical evaluation. It identifies weaknesses, risks, and potential failure points. Its purpose is not negativity but rather protection—ensuring the decision is realistically assessed.

Its focus typically includes:

* Questioning feasibility and identifying constraints
* Anticipating obstacles, liabilities, and unintended consequences
* Testing assumptions for fragility or over‑optimism
* Stress‑testing ideas under worst‑case scenarios

By exploring what could go wrong, the Black Hat strengthens decisions through rigorous scrutiny.

In [6]:
black_hat = black_hat_factory.BlackHatFactory.create(
    model=gemini_model
)
print("✅ Black Hat configured")

✅ Black Hat configured


### **4.4 Yellow — Optimism & Opportunity** 🟨

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#f1c40f" />
  <rect x="30" y="60" width="40" height="15" fill="#000000" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#f1c40f" />
</svg>

The Yellow Hat highlights the positive potential of an idea. It considers benefits, opportunities, and reasons an approach might succeed. This hat offers a constructive counterbalance to the Black Hat.

Areas of exploration include:

* Identifying advantages and potential gains
* Highlighting strategic or long‑term value
* Considering successful precedents or supportive evidence
* Exploring why an idea may be more viable than it initially appears

This perspective encourages balanced thinking by ensuring benefits are examined as carefully as risks.

In [7]:
yellow_hat = yellow_hat_factory.YellowHatFactory.create(
    model=gemini_model,
    search_model=gemini_model
)
print("✅ Yellow Hat configured")

✅ Yellow Hat configured


### **4.5 Green Hat — Creativity & Possibility**  🟩

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#2ecc71" />
  <rect x="30" y="60" width="40" height="15" fill="#000000" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#2ecc71" />
</svg>

The Green Hat invites imaginative thinking and exploration of alternatives. It pushes beyond conventional solutions and encourages innovation. Under this hat, criticism is suspended to allow ideas to emerge freely.

Typical contributions involve:

* Generating new ideas, variations, and alternatives
* Exploring unconventional approaches or re‑framing the problem
* Suggesting incremental improvements or radical shifts
* Encouraging divergent thinking without immediate evaluation

The Green Hat provides the creative fuel that opens pathways to novel, high‑value solutions.

In [8]:
green_hat = green_hat_factory.GreenHatFactory.create(
    model=gemini_model
)
print("✅ Green Hat configured")

✅ Green Hat configured


### **4.6 Blue — Structure & Synthesis**  🟦 

<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" width="100" height="100">
  <rect x="30" y="20" width="40" height="55" fill="#259deeff" />
  <rect x="30" y="60" width="40" height="15" fill="#000000" />
  <rect x="10" y="75" width="80" height="10" rx="2" fill="#259deeff" />
</svg>

The Blue Hat manages the entire thinking process. It defines goals, sequences the use of the hats, organizes insights, and synthesizes conclusions. Functioning like a facilitator, it ensures clarity and coherence.

Key responsibilities include:

* Setting the agenda and determining which hats to use and when
* Maintaining focus and preventing discussions from derailing
* Summarizing contributions from all hats
* Producing the final structured recommendation or decision

The Blue Hat transforms raw inputs into actionable, organized outcomes.

In [9]:
blue_hat = blue_hat_factory.BlueHatFactory.create(
    model=gemini_model
)
print("✅ Blue Hat configured")

✅ Blue Hat configured


## 5. Parallel Thinking Team 🤝

We create a `ParallelAgent` that runs the White, Red, Black, Yellow, and Green Hats concurrently on the same user prompt. This captures multiple perspectives in a single pass.

```mermaid
graph TB

    
    subgraph Step1Box[" Step 1: SixHatsBrainstorm"]
        direction TB
        
        ParallelBox[SixHatsBrainstorm<br/>Parallel Agent]
        
        ParallelBox -->|parallel| GreenHat[GreenHat Agent<br/>Creative Thinking]
        ParallelBox -->|parallel| YellowHat[YellowHat Agent<br/>Positive Thinking]
        ParallelBox -->|parallel| BlackHat[BlackHat Agent<br/>Critical Thinking]
        ParallelBox -->|parallel| RedHat[RedHat Agent<br/>Emotional Thinking]
        ParallelBox -->|parallel| WhiteHat[WhiteHat Agent<br/>Factual Thinking]
        
        YellowHat --> GoogleOptimist[google_optimist]
        YellowHat --> GetData[get_positive_data]
        RedHat --> GoogleSearch[google_search]
        WhiteHat --> GoogleSearch[google_search]
    end
    

    
    style ParallelBox fill:#475569,stroke:#e2e8f0,stroke-width:3px,color:#fff,rx:10,ry:10
    style GreenHat fill:#10b981,stroke:#059669,stroke-width:3px,color:#fff,rx:10,ry:10
    style YellowHat fill:#fbbf24,stroke:#d97706,stroke-width:3px,color:#000,rx:10,ry:10
    style BlackHat fill:#1f2937,stroke:#4b5563,stroke-width:3px,color:#fff,rx:10,ry:10
    style RedHat fill:#ef4444,stroke:#dc2626,stroke-width:3px,color:#fff,rx:10,ry:10
    style WhiteHat fill:#f8fafc,stroke:#64748b,stroke-width:3px,color:#000,rx:10,ry:10
    style GoogleOptimist fill:#374151,stroke:#6b7280,stroke-width:2px,color:#fff,rx:8,ry:8
    style GetData fill:#22c55e,stroke:#16a34a,stroke-width:2px,color:#fff,rx:8,ry:8
    style GoogleSearch fill:#374151,stroke:#6b7280,stroke-width:2px,color:#fff,rx:8,ry:8
    style Step1Box fill:#f1f5f9,stroke:#64748b,stroke-width:4px,rx:15,ry:15

```


In [10]:
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

print("✅ Thinking Team configured")

✅ Thinking Team configured


## 6. Orchestrator Workflow 🧬

```mermaid
graph TB

    Solver[SixHatsSolver<br/>Sequential Agent]
    

    
    Solver -->|connects to| Step1Box
    Solver -->|connects to| Step2Box
    
    subgraph Step1Box[" Step 1: SixHatsBrainstorm"]
        direction TB
        
        ParallelBox[Thinking Team<br/>Parallel Agent]
        

    end
    
    subgraph Step2Box[" Step 2: Judgment"]
        direction TB
        BlueHat[BlueHat Agent<br/>Synthesis & Decision]
    end
    
    Step1Box -.->|insights| Step2Box
    
    style Solver fill:#2d3748,stroke:#cbd5e1,stroke-width:4px,color:#fff,rx:10,ry:10
    style ParallelBox fill:#475569,stroke:#e2e8f0,stroke-width:3px,color:#fff,rx:10,ry:10
    style Step1Box fill:#f1f5f9,stroke:#64748b,stroke-width:4px,rx:15,ry:15
    style Step2Box fill:#f1f5f9,stroke:#64748b,stroke-width:4px,rx:15,ry:15
```

We define a `SequentialAgent` that first runs the parallel thinking team and then hands all aggregated outputs to the Blue Hat for synthesis and final decision-making.


In [11]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)

print("✅ Workflow configured")

✅ Workflow configured


### 6.1 Runner & Debug Execution 🧪

We use an `InMemoryRunner` with a logging plugin to execute the full Six Hats workflow on a sample decision question and inspect intermediate steps for debugging.

> 🧪 **Debug Mode:** `run_debug` exposes intermediate agent outputs, which is helpful for tracing how each hat influences the final decision.


In [12]:
runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

✅ Runner configured


### 6.2 Execute the runner and generate the responses

In [13]:
response = await runner.run_debug("Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?")


 ### Created new session: debug_session_id

User > Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-6a876c3a-1e35-4e85-b860-a85c44facb82
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: SixHatsSolver
[logging_plugin]    User Content: text: 'Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-6a876c3a-1e35-4e85-b860-a85c44facb82
[logging_plugin]    Starting Agent: SixHatsSolver
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsSolver
[logging_plugin]    Invocation ID: e-6a876c3a-1e35-4e85-b860-a85c44facb82
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsBrainstorm
[logging_plugin]    Invoc

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Token Usage - Input: 310, Output: 30
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: e09a73c0-d91c-4180-81c5-a7f74ff7a5e7
[logging_plugin]    Author: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['google_optimist']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: google_optimist
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Function Call ID: adk-58665653-f595-4de7-aa1c-5109ed5c4439
[logging_plugin]    Arguments: {'request': 'Pros and cons of switching backend database from PostgreSQL to NoSQL for a startup'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-a49aa687-84a1-4370-a754-7dbadadd1f2d
[logging_plugin]    Session ID: 15dc7f43-fd2c-4fdd-9367-de6b986

## 7. Blue Hat 🟦 Final Recommendation 📋

Finally, we render the Blue Hat’s synthesized recommendation as rich Markdown, providing a structured decision summary that incorporates all five thinking perspectives.


In [15]:
# Final Recommendation
display(Markdown(response[-1].content.parts[0].text))

Alright team, let's bring our focus together. We're at a critical juncture, deciding whether to migrate our backend database from PostgreSQL to a NoSQL solution. This decision has significant implications for our startup's development velocity, scalability, and data integrity.

**Our Objective:** To make an informed decision about migrating from PostgreSQL to a NoSQL database, weighing the potential benefits against the risks and challenges, and ensuring it aligns with our startup's current and future needs.

We've heard a range of perspectives, from the cautious warnings to the enthusiastic visions. Let's consolidate our findings to pave the way for a clear path forward.

Here's a summary of what we've gathered:

**[WhiteHatAgent]'s Objective Analysis:**
*   **NoSQL Strengths:** Designed for unstructured/semi-structured data, flexible schemas, excels at horizontal scalability for massive data/high throughput, often faster write speeds, prioritizes availability/performance over immediate consistency (eventual consistency is common).
*   **NoSQL Limitations:** Lacks standardized query language (no SQL), complex joins are difficult, ACID compliance is often absent or weakened, query limitations, data integrity must be managed at the application level.
*   **PostgreSQL Strengths:** Robust relational database, strong ACID compliance, excellent for structured data and complex queries (joins, aggregations), mature ecosystem, evolving with JSON support and extensions (e.g., vector search), primarily scales vertically.
*   **PostgreSQL Limitations:** Native horizontal scaling is complex, not inherently designed for massive analytics workloads, fault tolerance/multi-region replication can be complex.
*   **Information Gaps:** Specific startup data requirements, current PostgreSQL pain points, team expertise, precise scalability needs, query complexity, and consistency requirements are not detailed.

**[YellowHatAgent]'s Optimistic Outlook:**
*   NoSQL offers supercharged scalability and flexibility, ideal for rapid growth and handling massive data.
*   Flexible schemas enable lightning-fast iteration and adaptation, perfect for agile startup environments.
*   PostgreSQL has incredible strengths: rock-solid data integrity (ACID), complex query handling, and evolving features (JSON, AI extensions) that can meet modern needs.
*   A hybrid approach can leverage the best of both worlds. The key is aligning the solution with specific data structures, access patterns, and growth aspirations.

**[RedHatAgent]'s Cautious Assessment:**
*   Expresses strong concern about switching from a reliable PostgreSQL to potentially complex NoSQL.
*   Worries about unforeseen headaches, complexity, and loss of stability.
*   Stresses the importance of sticking with what's known and trusted, at least for now.

**[BlackHatAgent]'s Risk-Focused Analysis:**
*   **Core Question:** What *specific* problem are we solving with NoSQL that PostgreSQL *cannot* handle effectively?
*   **Key Risks:** Data consistency nightmares (stale data, conflicts), illusion of schema flexibility leading to application-level chaos and debugging hell, weak or absent transaction support (ACID), querying complexity for non-trivial relationships, significant operational overhead and expertise required for distributed systems, fragmented tooling and potential vendor lock-in.
*   Strongly advises against chasing shiny objects without clear, demonstrable benefits outweighing significant risks.

**[GreenHatAgent]'s Visionary Ideas (Focus on potential of flexible data structures):**
*   Envisions data as a "living organism," "Play-Doh," or "cosmic dust cloud," adaptable and self-organizing.
*   Suggests dynamic data generation, predictive data oracles, and "telepathic" librarians.
*   These ideas, while creative, highlight the *potential* for extreme data flexibility and adaptability that NoSQL can represent, moving beyond rigid structures.

---

**Synthesis and Next Steps:**

We have a clear dichotomy: the proven reliability and structured power of PostgreSQL versus the purported flexibility and scalability of NoSQL. The core concern raised by Black Hat and Red Hat, echoed by the need for specific data in White Hat, is whether the pain points of our current system (if any) are severe enough to warrant the significant risks and complexities of a NoSQL migration.

Yellow Hat and Green Hat highlight the *potential* benefits, particularly around agility and handling varied data, but we need to ground these potentials in our specific reality.

Before we can make a decision, we need to fill those crucial **Information Gaps** identified by White Hat. Therefore, my proposed next steps are:

1.  **Deep Dive into Current Pain Points:** The product and engineering leads need to formally document the *specific* challenges and limitations we are experiencing with PostgreSQL *today*. Are we hitting performance ceilings? Are certain data structures proving exceptionally difficult? Is schema evolution a constant bottleneck?
2.  **Define Future Data Needs & Scalability Projections:** We need concrete projections:
    *   What types of data are we expecting to store and how will they evolve? (Structured, semi-structured, unstructured?)
    *   What are our anticipated growth rates for users and data volume over the next 1-3 years?
    *   What are our critical transaction requirements? (e.g., financial, inventory, user authentication). What is our tolerance for data inconsistency?
3.  **Assess Team Expertise:** An honest evaluation of our team's proficiency with SQL vs. various NoSQL paradigms. What would the learning curve and training investment look like?
4.  **Analyze Query Patterns:** A detailed breakdown of the most critical and complex queries our application performs, and how frequently.
5.  **Explore PostgreSQL's Evolving Capabilities:** Before dismissing PostgreSQL, let's identify specific use cases where extensions or advanced features (like JSONB, pgvector) could potentially solve our "NoSQL-like" needs within our current relational framework.

Once we have this detailed information, we can reconvene. At that point, we can take the following steps:

*   **Evaluate Specific NoSQL Options:** If the data points to a need for NoSQL, we'll evaluate specific solutions (e.g., MongoDB, Cassandra, DynamoDB) based on their strengths and weaknesses relative to our identified needs.
*   **Consider Hybrid Approaches:** Assess if a hybrid model, using PostgreSQL for core transactional data and a NoSQL solution for specific high-volume, less structured data, is feasible and beneficial.
*   **Risk Mitigation Plan:** For any proposed migration, develop a robust risk mitigation strategy addressing consistency, operational overhead, and data integrity.

**Decision Point:** For now, the answer is **"We need more information."** We are not ready to commit to a migration. Our immediate focus must be on gathering the detailed data outlined above.

Does this plan sound like a clear and logical way forward?